# GCP リソース作成

## パラメータ設定

In [1]:
vpc_name="topse-vpc"
subnet_name="topse-subnet"
region="us-west1"
ip_range="10.1.0.0/20"

## VPC作成

In [ ]:
!gcloud compute networks create {vpc_name} --description="topse custom network" --subnet-mode=custom

## サブネット作成

In [2]:
!gcloud compute networks subnets create {subnet_name} --network={vpc_name} --region={region} --range={ip_range}

ERROR: (gcloud.compute.networks.subnets.create) Could not fetch resource:
 - The resource 'projects/topse-nagaku/regions/us-west1/subnetworks/topse-subnet' already exists



RuntimeError: Unexpected exit code: 1

## ファイアウォールルール作成

In [3]:
!gcloud compute firewall-rules create {vpc_name}-ssh-ingress --description="general firewall to allow ssh for all instances in {vpc_name}" \
--direction=INGRESS --priority=1000 --network={vpc_name} --action=ALLOW --rules=tcp:22
!gcloud compute firewall-rules create {vpc_name}-mysql-ingress --description="firewall to allow mysql for wordpress instances in {vpc_name}" \
--direction=INGRESS --priority=1000 --network={vpc_name} --action=ALLOW --rules=tcp:3306 --source-ranges={ip_range}

Creating firewall...failed.                                                    
ERROR: (gcloud.compute.firewall-rules.create) Could not fetch resource:
 - The resource 'projects/topse-nagaku/global/firewalls/topse-vpc-ssh-ingress' already exists



RuntimeError: Unexpected exit code: 1

## GKE作成

In [4]:
cluster_name = "topse-cluster"
zones = ["us-west1-a","us-west1-b"]
!gcloud container clusters create {cluster_name} \
--region {region} \
--node-locations  {zones[0]},{zones[1]} \
--enable-ip-alias \
--network {vpc_name} \
--subnetwork {subnet_name} \
--num-nodes=1

This will enable the autorepair feature for nodes. Please see https://cloud.google.com/kubernetes-engine/docs/node-auto-repair for more information on node autorepairs.
Creating cluster topse-cluster in us-west1... Cluster is being health-checked..
.⠧                                                                             
Creating cluster topse-cluster in us-west1... Cluster is being health-checked (
master is healthy)...done.                                                     
Created [https://container.googleapis.com/v1/projects/topse-nagaku/zones/us-west1/clusters/topse-cluster].
To inspect the contents of your cluster, go to: https://console.cloud.google.com/kubernetes/workload_/gcloud/us-west1/topse-cluster?project=topse-nagaku
kubeconfig entry generated for topse-cluster.
NAME           LOCATION  MASTER_VERSION  MASTER_IP       MACHINE_TYPE   NODE_VERSION    NUM_NODES  STATUS
topse-cluster  us-west1  1.13.11-gke.14  35.227.145.133  n1-standard-1  1.13.11-gke.14  2          

## kubectl設定

In [5]:
!gcloud container clusters get-credentials {cluster_name} --region {region}
!curl -LO https://storage.googleapis.com/kubernetes-release/release/$(curl -s https://storage.googleapis.com/kubernetes-release/release/stable.txt)/bin/linux/amd64/kubectl
!chmod +x ./kubectl
!sudo mv ./kubectl /usr/local/bin/kubectl
!kubectl version

Fetching cluster endpoint and auth data.
kubeconfig entry generated for topse-cluster.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 41.4M  100 41.4M    0     0   129M      0 --:--:-- --:--:-- --:--:--  129M
Client Version: version.Info{Major:"1", Minor:"17", GitVersion:"v1.17.0", GitCommit:"70132b0f130acc0bed193d9ba59dd186f0e634cf", GitTreeState:"clean", BuildDate:"2019-12-07T21:20:10Z", GoVersion:"go1.13.4", Compiler:"gc", Platform:"linux/amd64"}
Server Version: version.Info{Major:"1", Minor:"13+", GitVersion:"v1.13.11-gke.14", GitCommit:"56d89863d1033f9668ddd6e1c1aea81cd846ef88", GitTreeState:"clean", BuildDate:"2019-11-07T19:12:22Z", GoVersion:"go1.12.11b4", Compiler:"gc", Platform:"linux/amd64"}


## VPC Peering作成

### パラメータ設定

In [ ]:
vpc_peering_name="topse-vpc-peering"

### VPC Peering作成

In [ ]:
!gcloud compute addresses create {vpc_peering_name} \
    --global \
    --purpose=VPC_PEERING \
    --prefix-length=16 \
    --description="for VPC peering to gcp service" \
    --network={vpc_name}

以下の gcloud services vpc-peerings update がエラーになった場合、エラーメッセージの案内に従って、[APIとサービス] からCloud Resource Manager APIを有効にする。

In [ ]:
!gcloud services vpc-peerings update \
    --service=servicenetworking.googleapis.com \
    --ranges={vpc_peering_name} --network={vpc_name} \
    --force --quiet

## Cloud SQL作成

### パラメータ設定

In [15]:
# need to change instance name uniquely even if instance is deleted. The instance name is remaining for a while
import datetime
suffix = format(datetime.date.today(), '%Y%m%d')
cloud_sql_name = "topse-wordpress-mysql-" + suffix
database_version = "MYSQL_5_7"
root_password = "8sg3jJguHLq8KjNk"
print(cloud_sql_name)

topse-wordpress-mysql-20200130


### Cloud SQL作成

In [6]:
!gcloud beta sql instances create {cloud_sql_name} --no-assign-ip --network={vpc_name} --no-backup --database-version={database_version} \
--root-password={root_password} --region {region}

Creating Cloud SQL instance...done.                                            
Created [https://www.googleapis.com/sql/v1beta4/projects/topse-nagaku/instances/topse-wordpress-mysql-03].
NAME                      DATABASE_VERSION  LOCATION    TIER              PRIMARY_ADDRESS  PRIVATE_ADDRESS  STATUS
topse-wordpress-mysql-03  MYSQL_5_7         us-west1-a  db-n1-standard-1  -                172.24.0.3       RUNNABLE


### IPアドレス取得

In [ ]:
output=!gcloud beta sql instances describe {cloud_sql_name} --format json
import json
while output[0].find('{') == -1:
    del output[0]
output[0] = "{"
json_result = json.loads(''.join(output))
print(json_result["ipAddresses"][0]["ipAddress"])
cloud_sql_ip=json_result["ipAddresses"][0]["ipAddress"]

### Secret作成

MySQLのrootパスワードをKubernatesのSecretに登録する

In [33]:
!kubectl create secret generic mysql --from-literal=password={root_password}

secret/mysql created


## Cloud DNS作成

### パラメータ設定

In [2]:
private_zone_name="topse-private-zone"
dns_name="topse.private."

### プライベートゾーン作成

In [ ]:
!gcloud beta dns managed-zones create {private_zone_name} --dns-name={dns_name} \
--description={private_zone_name} --visibility=private --networks {vpc_name}

### DNSレコード追加

Cloud SQLのIPアドレス、ホスト名を登録する。

In [19]:
#!gcloud dns record-sets transaction abort --zone={private_zone_name}
!gcloud dns record-sets transaction start --zone={private_zone_name}
!gcloud dns record-sets transaction add {cloud_sql_ip} --name=mysql.{dns_name} --ttl=300 --type=A --zone={private_zone_name}
!gcloud dns record-sets transaction execute --zone={private_zone_name}

Transaction started [transaction.yaml].
Record addition appended to transaction at [transaction.yaml].
Executed transaction [transaction.yaml] for managed-zone [topse-private-zone].
Created [https://dns.googleapis.com/dns/v1/projects/topse-nagaku/managedZones/topse-private-zone/changes/1].
ID  START_TIME                STATUS
1   2020-01-13T14:34:29.166Z  pending


## FileStore作成

### パラメータ設定

In [21]:
filestore_name="nfs-server"
volume_capacity="1TB"

Waiting for [operation-1578926090150-59c0660bae265-7a91f3ba-0811f466] to finish
...done.                                                                       


### FileStore作成

In [ ]:
!gcloud beta filestore instances create {filestore_name} --zone {zones[0]} \
--tier=STANDARD --file-share=name="vol1",capacity={volume_capacity} --network=name={vpc_name}

### IPアドレス取得

In [29]:
output=!gcloud beta filestore instances describe {filestore_name} --zone {zones[0]} --format json
import json
while output[0].find('{') == -1:
    del output[0]
output[0] = "{"
json_result = json.loads(''.join(output))
print(json_result["networks"][0]["ipAddresses"][0])
filestore_ip=json_result["networks"][0]["ipAddresses"][0]

10.46.200.226


### DNSレコード追加

FileStoreのIPアドレス、ホスト名を登録する。

In [30]:
#!gcloud dns record-sets transaction abort --zone={private_zone_name}
!gcloud dns record-sets transaction start --zone={private_zone_name}
!gcloud dns record-sets transaction add {filestore_ip} --name=nfs.{dns_name} --ttl=300 --type=A --zone={private_zone_name}
!gcloud dns record-sets transaction execute --zone={private_zone_name}

Transaction started [transaction.yaml].
Record addition appended to transaction at [transaction.yaml].
Executed transaction [transaction.yaml] for managed-zone [topse-private-zone].
Created [https://dns.googleapis.com/dns/v1/projects/topse-nagaku/managedZones/topse-private-zone/changes/2].
ID  START_TIME                STATUS
2   2020-01-13T14:40:51.697Z  pending


# GCPリソース削除

## Cloud DNS削除

In [53]:
!gcloud dns record-sets transaction start --zone={private_zone_name}
!gcloud dns record-sets transaction remove {cloud_sql_ip} --name=mysql.{dns_name} --ttl=300 --type=A --zone={private_zone_name}
!gcloud dns record-sets transaction remove {filestore_ip} --name=nfs.{dns_name} --ttl=300 --type=A --zone={private_zone_name}
!gcloud dns record-sets transaction execute --zone={private_zone_name}
!gcloud beta dns managed-zones delete {private_zone_name}

Transaction started [transaction.yaml].
Record removal appended to transaction at [transaction.yaml].
Record removal appended to transaction at [transaction.yaml].
Executed transaction [transaction.yaml] for managed-zone [topse-private-zone].
Created [https://dns.googleapis.com/dns/v1/projects/topse-nagaku/managedZones/topse-private-zone/changes/3].
ID  START_TIME                STATUS
3   2020-01-13T15:20:03.393Z  pending
Deleted [https://dns.googleapis.com/dns/v1beta2/projects/topse-nagaku/managedZones/topse-private-zone].


## FileStore削除

In [ ]:
!gcloud beta filestore instances delete {filestore_name} --zone {zones[0]} --quiet

## CloudSQL削除

In [ ]:
!gcloud sql instances delete {cloud_sql_name} --quiet

## VPC Peering削除

In [ ]:
!gcloud compute addresses delete {vpc_peering_name} --global --quiet

## Secret削除

In [48]:
!kubectl delete secret mysql

secret "mysql" deleted


## GKE削除

In [55]:
!gcloud container clusters delete {cluster_name} --region {region} --quiet

Waiting for [operation-1578929223330-59c071b7b692f-62ed182f-b74db7db] to finish
...done.                                                                       
Deleting Cloud SQL instance...done.                                            
Deleted [https://www.googleapis.com/sql/v1beta4/projects/topse-nagaku/instances/topse-wordpress-mysql-03].
Deleted [https://www.googleapis.com/compute/v1/projects/topse-nagaku/global/firewalls/topse-vpc-ssh-ingress].
Deleted [https://www.googleapis.com/compute/v1/projects/topse-nagaku/global/firewalls/topse-vpc-mysql-ingress].
Deleted [https://www.googleapis.com/compute/v1/projects/topse-nagaku/regions/us-west1/subnetworks/topse-subnet].


## ファイアーウォールルール削除

In [ ]:
!gcloud compute firewall-rules delete {vpc_name}-ssh-ingress --quiet
!gcloud compute firewall-rules delete {vpc_name}-mysql-ingress --quiet

## サブネット削除

In [ ]:
!gcloud compute networks subnets delete {subnet_name} --region={region} --quiet

## VPC削除

In [ ]:
!gcloud compute networks delete {vpc_name} --quiet